In [176]:
# Импорт необходимых библиотек
import pandas as pd
import sqlite3
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import smtplib

from sqlalchemy import sql

In [82]:
# Создайте базу данных sqlite, можно использовать sqlalchemy или pandas, или любой другой инструмент
# Задайте почту на которую будет отправлен отчет (создайте свою по примеру ранее, или воспользуйтесь готовой)

CONN = sqlite3.connect('currency.db')
EMAIL = 'etltest22-03@yandex.ru'
CRED = ("etltest22-03", "etltest22-03pass1")
HOST = "smtp.yandex.ru"
TO = EMAIL
FROM = EMAIL

In [97]:
# Напишите функцию которая принимает на вход дату
# и выгружает курс валюты EUR к USD за этот день
# Функция должна вернуть список словарей или можно использовать можно использоват pandas.read_csv()

# Для этого воспользуйтесь https://exchangerate.host/#/
# и методом Timeseries Endpoint! Строго им. Где start_date = end_date = date
# Используйте параметры start_date, end_date, base=EUR, symbols=USD, format=csv

def extract_currency(date,symbols):
    URL = f'https://api.exchangerate.host/timeseries?start_date={date}&end_date={date}&symbols={symbols}&format=csv'
    return pd.read_csv(URL).drop(columns=['start_date', 'end_date']).rename(columns={'code': 'currency'})

In [98]:
# Напишите функцию которая скачивает данные с гитхаба
# Функция принимает на вход дату
# Функция должна вернуть список словарей или можно использовать можно использоват pandas.read_csv()

# ссылка на гитхаб с файлами - https://github.com/dm-novikov/stepik_airflow_course/tree/main/data_new

def extract_data(date):
    URL = f"https://raw.githubusercontent.com/dm-novikov/stepik_airflow_course/main/data_new/{date}.csv"
    return pd.read_csv(URL)

In [99]:
date = '2021-01-01'
currency = extract_currency(date, 'USD')
data = extract_data(date)

In [130]:
insert_to_db(currency, 'currency', CONN)
insert_to_db(data, 'data', CONN)

In [128]:
sql_query('DROP table currency', CONN)
sql_query('DROP table data', CONN)

In [164]:
sql_query('SELECT data.date, value as eur, rate as  usd FROM data JOIN currency ON data.date=currency.date',CONN)

,date,eur,usd
0,2021-01-01,38,"1,217582"
1,2021-01-01,65,"1,217582"
2,2021-01-01,74,"1,217582"
3,2021-01-01,42,"1,217582"
4,2021-01-01,23,"1,217582"
5,2021-01-01,48,"1,217582"
6,2021-01-01,86,"1,217582"
7,2021-01-01,74,"1,217582"
8,2021-01-01,24,"1,217582"


In [68]:
# Напишите функцию которая принимает на вход список словарей
# и загружает в табличку в sqlite, можно использоват pandas.to_sql()

def insert_to_db(data, table_name, conn):
    data.to_sql(name=table_name,if_exists='append', con=conn)

In [106]:
# Напишите функцию которая выполнит SQL запрос к базу данных
# Здесь удобно использовать sqlalchemy или https://pythonru.com/biblioteki/vvedenie-v-sqlite-python
# Если выполняется select ... то функция должна возвращать результат (не None)
# Если используется внутренняя операция например truncate table, insert into select ... то возвращает None
# data = sql_query(sql="select * from table") -> [(1, 2, 3), (4, 5, 6) ...]
# sql_query("truncate table") -> None

def sql_query(sql, conn):
    command = sql.split()[0]
    cursor = conn.cursor()
    cursor.execute(sql)
    if command.lower() == "select":
        return pd.read_sql_query(sql, conn)

In [165]:
# Напишите функцию которая отправит полученный отчет на почту
# Для этого можете воспользоваться кодом который был написан ранее (на уроке)
# обернув его в вашу функцию
def html_pretty(df):
    """ Pretty html dataframe
    """
    return """\
    <html>
      <head></head>
      <body>
        {0}
      </body>
    </html>
    """.format(df.to_html())


def _send_email(data, cred, host, port, to, From):
    """ Send DF to email
    """

    msg = MIMEMultipart()
    part = MIMEText(html_pretty(data), 'html')
    msg.attach(part)

    server = smtplib.SMTP(host, port)
    server.starttls()
    server.login(cred[0], cred[1])
    server.sendmail(From, to, msg.as_string())
    server.quit()
  

def send_report(data, cred, host, to, From, port):
    _send_email(data=data, cred=cred, host=host, to=to, From=From, port=port)

In [190]:
# Запустите ваш код в функции main

# Напишите генерацию дат, так чтобы у вас получился список
# 2021-01-02, 2021-01-03 ... etc
# Нужны даты с 2021-01-01 по 2021-01-04

dates_list = list(['2021-01-0'+str(date) for date in range(1,5)])

def main(date, email, conn):
  
  # Выгружаем данные по валютам и из источника
    currency = extract_currency(date, 'USD')
    data = extract_data(date)

  # Вставляем данные в БД
  # У вас долнжо получиться 2 таблицы CURRENCY, DATA
    insert_to_db(currency, 'currency', conn)
    insert_to_db(data, 'data', conn)

  # Объединение данных в отдельную таблицу JOIN_DATA
  # CURRENCY, DATA объединить через JOIN по дате и валюте
  # Используйте insert into select ... inner join ...
  # И соответственно создать таблицу для этого
    join_data = sql_query('SELECT data.date as date, value as eur, rate as usd FROM data INNER JOIN currency ON data.date=currency.date',conn)
    insert_to_db(data, 'join_data', conn)
    
  # Очистка временных таблиц CURRENCY, DATA
  # Это те таблицы куда произошла вставка данных
  # Используйте truncate
    sql_query('DROP table currency', conn)
    sql_query('DROP table data', conn)

  # Получение данных из таблицы JOIN_DATA за конкретный день
  # Используйте date при формировании запроса select * from ... where date ='...'
    report = sql_query(f'SELECT * FROM join_data WHERE date="{date}"', conn)

  # Отправка данных на почту
  # Передайте данные в любом виде и отправьте на почту
    send_report(report, cred=CRED, host=HOST, port=587, to=email, From=email)


# Пройдемся по списку дат и выполним скрипт
for date in dates_list:
    main(date, EMAIL, CONN)